In [4]:
from sklearn.datasets import load_iris
import RandomForest as RF
import os.path
import os
from datetime import datetime
import bz2
import _pickle as cPickle
import numpy as np
import random
import warnings
warnings.filterwarnings("ignore")

my_os = "mac" #win mac

separator = "\\"

if my_os == "mac":
    separator = "//"

dp = "5"
today = datetime.now()
folderName = 'Experiments 3 DP ' + dp + ' '  + today.strftime("%d-%b-%Y %H_%M_%S")

if not os.path.exists(folderName):
    os.makedirs(folderName)

dataset = "unsw_nb15" # iris kdd  nsl_kdd  unsw_nb15 cic_ids_2017
problem = "AC" # AD AC

criterion_input = "entropy" #["gini", "entropy"]
ensemble_method = "SV"
folder = "Experiments 2 subsets DP " + dp

method = ""
#RF_S_DTs_A_All RF_S_DTs_WA_All RF_S_DTs_A RF_S_DTs_WA
trees = -1


if dataset == "kdd" and problem == "AD" :
    # num_trees = 65
    criterion_input = 'entropy' #["gini", "entropy"]
    ensemble_method = "SV"
    trees = 129
    method = "RF_S_DTs_A_All"

if dataset == "kdd" and problem == "AC" :
    # num_trees = 53
    criterion_input = 'entropy' #["gini", "entropy"]
    ensemble_method = "WV"
    trees = 3
    method = "RF_S_DTs_WA_All"

if dataset == "nsl_kdd" and problem == "AD" :
    # num_trees = 65
    criterion_input = 'gini' #["gini", "entropy"]
    ensemble_method = "SV"
    trees = 129
    method = "RF_S_DTs_A_All"

if dataset == "nsl_kdd" and problem == "AC" :
    # num_trees = 31
    criterion_input = 'entropy' #["gini", "entropy"]
    ensemble_method = "SV"
    trees = 62
    method = "RF_S_DTs_WA_All"

if dataset == "unsw_nb15" and problem == "AD" :
    # num_trees = 93
    criterion_input = 'entropy' #["gini", "entropy"]
    ensemble_method = "WV"
    trees = 483
    method = "RF_S_DTs_WA_All"

if dataset == "unsw_nb15" and problem == "AC" :
    # num_trees = 61
    criterion_input = 'entropy' #["gini", "entropy"]
    ensemble_method = "SV"
    trees = 123
    method = "RF_S_DTs_WA_All"

if dataset == "cic_ids_2017" and problem == "AD" :
    criterion_input = 'entropy' #["gini", "entropy"]
    ensemble_method = "WV"
    # num_trees = 77
    trees = 70
    method = "RF_S_DTs_A"

if dataset == "cic_ids_2017" and problem == "AC" :
    criterion_input = 'entropy' #["gini", "entropy"]
    ensemble_method = "WV"
    # num_trees = 95
    trees = 14
    method = "RF_S_DTs_A"

no_runs = 5
start_from_run = 1

k = start_from_run

file_name_data_same = folder + separator +dataset+ separator + dataset +"_"+ problem + "_"
file_name_algorithms_same  = folder + separator +dataset+ separator + "Experiment_2_2_RF_"+dataset+"_"+problem+"_"+criterion_input+"_"+ensemble_method+"_"


wrong = 0
while k<=no_runs:
    try:
        file_name_data = file_name_data_same + str(k) + "_subsets_for_Experiment3.pbz2"
        f = bz2.BZ2File(file_name_data, "rb")
    except FileNotFoundError:
        wrong+=1
    k+=1

if wrong>0:
    print("Error: Make sure that you have the folder with results from Experiment 2 with the following path: 'Experiments 2 subsets DP [epsilon]//[dataset]//' and that it contains one file for each of 'k' runs with the following naming convention: '[dataset]_[AD/AC]_[k]_subsets_for_Experiment3.pbz2'")
else:
    k = start_from_run
    sum_acc = 0
    sum_f1 = 0
    result = ""
    while k<=no_runs:

        file_name_data = file_name_data_same + str(k) + "_subsets_for_Experiment3.pbz2"
        f = bz2.BZ2File(file_name_data, "rb")
        dataTraining, targetTraining, subsetsTraining, dataValidation, targetValidation, subsetsValidation, dataTesting, targetTesting, subsetsTesting = cPickle.load(f)
        f.close()

        print("\n INFORMATION\n")
        print("Dataset: "+dataset)
        print("Number Training Cases: "+str(dataTraining.shape[0]))
        print("Number whole Testing Cases: "+str(dataTesting.shape[0])+"\n")


        file_name_algorithms = file_name_algorithms_same + str(k)+".pbz2"
        #f = open(file_name, "rb")
        f = bz2.BZ2File(file_name_algorithms, "rb")
        algorithms = cPickle.load(f)
        f.close()

        print("**********************************************")
        print()
        print("            EXPERIMENT 3")
        print()
        print("**********************************************")

        print("   RUN " + str(k))

        numTreesPerRF = trees
        fl = RF.FederatedLearning()
        # mergeMethod2 mergeMethod3 mergeMethod4
        if method == "RF_S_DTs_A_All":
            fl.mergeMethod1(algorithms, numTreesPerRF)
        if method == "RF_S_DTs_WA_All":
            fl.mergeMethod2(algorithms, numTreesPerRF)
        if method == "RF_S_DTs_A":
            fl.mergeMethod3(algorithms, numTreesPerRF)
        if method == "RF_S_DTs_WA":
            fl.mergeMethod4(algorithms, numTreesPerRF)

        fl.predict(dataTesting, targetTesting)

        print("numTrees: "+str(fl.size)+"     Accuracy: " + str(fl.statisticsRF.accuracy)  + " F1: "  + str(fl.statisticsRF.f1))

        result += str(k) + "Accuracy: " + str(fl.statisticsRF.accuracy) + " F1: " + str(fl.statisticsRF.f1) + "\n"

        sum_acc += fl.statisticsRF.accuracy
        sum_f1 += fl.statisticsRF.f1

        f = bz2.BZ2File(folderName+separator+"Experiment_3_RF_"+dataset+"_"+problem+"_"+criterion_input+"_"+ensemble_method+"_"+method + "_"+str(k)+".pbz2", "wb")
        cPickle.dump((fl),f)
        f.close()


        k+=1

    avg_acc = sum_acc/no_runs
    avg_f1 = sum_f1/no_runs
    result += "Average accuracy: " + str(avg_acc) + "\n"
    result += "Average f1: " + str(avg_f1) + "\n"

    f1 = open(folderName + separator+ "DP_" + dataset + "_" + problem + "_" + method + "_" + str(trees) + ".txt", "w")
    f1.write(result)
    f1.close()


 INFORMATION

Dataset: unsw_nb15
Number Training Cases: 55947
Number whole Testing Cases: 15985

**********************************************

            EXPERIMENT 3

**********************************************
   RUN 1
numTrees: 123     Accuracy: 82.139505786675 F1: 0.8270148583986522

 INFORMATION

Dataset: unsw_nb15
Number Training Cases: 55947
Number whole Testing Cases: 15985

**********************************************

            EXPERIMENT 3

**********************************************
   RUN 2


KeyboardInterrupt: 